# Introduction

I came across this problem on leetcode.com. It's problem 753 'Cracking the Safe' (https://leetcode.com/problems/cracking-the-safe/). I copy the problem statement here for convenience.

> There is a box protected by a password. The password is a sequence of n digits where each digit can be one of the first k digits 0, 1, ..., k-1.
>
> While entering a password, the last n digits entered will automatically be matched against the correct password.
>
> For example, assuming the correct password is "345", if you type "012345", the box will open because the correct password matches the suffix of the entered password.
>
> Return any password of minimum length that is guaranteed to open the box at some point of entering it."

Here is an example of input and output to the required program from the same problem page.

> Input: n = 2, k = 2
> Output: "00110"
> Note: "01100", "10011", "11001" will be accepted too.

In that example you can easily see that the string '00110' contains all products tuples.

In [2]:
from itertools import product

[x for x in product([0,1], repeat=2

[(0, 0), (0, 1), (1, 0), (1, 1)]

Having solved the problem and becoming interested in the theory of it I discovered that the problem is equivalent to finding a De Bruijn sequence for the given alphabet (integers up to k) of order n.

Another worthy historical note is that De Bruijn himself noted the important contribution  to this work of two other mathematicians https://en.wikipedia.org/wiki/De_Bruijn_sequence. 

> According to De Bruijn himself [3] , the existence of De Bruijn sequences were first proved, for the case of alphabets with two elements, by Camille Flye Sainte-Marie in 1894, whereas the generalization to larger alphabets is originally due to Tanja van Ardenne-Ehrenfest [4] and himself.

My solution is given here as the function below and it keeps the nomenclature of the variables from the leetcode problem.

In [6]:

def crackSafe(n: int, k: int) -> str:

    if n == 1:
        return ''.join((str(i) for i in range(k)))

    todo = k**n
    done = 1
    s = '0'*n
    pres_to_suffs = {}
    while done < todo:
        prefix = s[-(n-1):]
        last_suff = pres_to_suffs.get(prefix, k)
        suff = last_suff - 1
        pres_to_suffs[prefix] = suff
        s += str(suff)
        done += 1
    return s
            

.. and a quick test

In [7]:
crackSafe(2, 2)

'00110'

In [8]:
crackSafe(3, 3)

'00022212202112102012001110100'

### How does it work?

First the algorithm starts with the 'first' product-tuple lexicographicall i.e. in crackSafe(3,3) that is '000'. We know that this product-tuple must exist in the sequence in any case.

We know that the next item in the sequence must have some suffix of the last product-tuple we have covered as a prefix. In the best case this is the last n-1 elements (for '000' that is obviously '00') being used as the first n-1 elements of the next. Part of De Bruijn's work was proving that you can always reuse n-1 elements. So for our next product tuple we must decide on some sequence '00*' where `*` is undecided and we have now 2 choices for it (presuming that we don't need to repeat '000').

The algorithm always chooses the highest value for the suffix that hasn't yet been chosen and remembers that choice in the the pres_to_suffs dictionary. In our example that is '2' meaning we choose the product-tuple '002'. The next time the '00' prefix is encountered we will choose '1' meaning we choose the product-tuple '001'. It is sufficient that we always choose items in this order.

Below is laid out a longer sequence of choices

... start with 000 -> output = '000'
next prefix = '00', highest unchosen suffix = 2, output ='0002'
next prefix = '02', highest unchosen suffix = 2, output ='00022'
next prefix = '22', highest unchosen suffix = 2, output ='000222'
next prefix = '22', highest unchosen suffix = 1, output ='0002221'
next prefix = '21', highest unchosen suffix = 2, output ='00022212'
next prefix = '12', highest unchosen suffix = 2, output ='000222122'
.. and so on

Having the memory in pres_to_suffs obviously allows us never to repeat a sequence. 

The strict ordering on choices is the next missing piece that allows the De Bruijn sequence to manifest itself. However the starting product-tuple must be chosen carefully. For example, instead of choosing the highest unchosen suffix we could choose the lowest unchosen suffix. This would not work if the first product-tuple was lexicographically the lowest. Specifically we would start with '000' and the resulting sequence would follow. 

```
000
0001
00010
000100
0001002
00010020
000100200
```

At this point we have seen 000, 001 and 002. There are no further unused suffixes for '00' and so we would have to repeat.

A correct starting product-tuple for this strict-ordering is the lexicographically highest one i.e. '222'. The code below demostrates.

In [17]:
def crackSafe2(n: int, k: int) -> str:

    if n == 1:
        return ''.join((str(i) for i in range(k)))

    todo = k**n
    done = 1
    s = str(k-1)*n
    pres_to_suffs = {}
    while done < todo:
        prefix = s[-(n-1):]
        last_suff = pres_to_suffs.get(prefix, -1)
        suff = last_suff + 1
        pres_to_suffs[prefix] = suff
        s += str(suff)
        done += 1
    return s
            
crackSafe2(3,3)

'22200010020110120210221112122'

### Questions Conjectures etc.

The total number of DeBruijn sequences is given by:
```
(k!) ** (k**n-1) / k**n
# k: size of alphabet
# n: length of sequences
``` 

Given an alphabet of size `n`, we have `n!` strict orderings (every permutation). We can conjecture that there is at least one good starting tuple for each strict ordering and so this gives us at least `n!` De Brujn sequences. In this model of thought, any additional De Brujn sequences would be due to non-strict-orderings or due to there being more than one good starting tuple for each strict ordering.

Is there at least one good starting tuple for each strict ordering?

Are there more than one good starting tuple for each strict ordering?

You can imagine that perhaps ordering 1,2,3 is the same as 3,2,1 (a wraparound shift). However this is not necessarily so as you are not guaranteed to use all the '1's before moving onto '2's etc. Is there ever a case where ordering 3,2,1 is the same as 1,2,3?

DeBruijn sequences wrap around, so you can start the same DeBruijn sequence in a different place and in that context generate the sequence while violating any strict ordering.

Can you find some starting point for any DeBruijn sequence that represents a strict ordering of suffixes?

